In [34]:
import pandas as pd
df = pd.read_csv('product.csv')
df_cauhinh_split = df['Cấu hình'].str.split(';', expand=True)
a = df_cauhinh_split[0][0].find('[')
df_cauhinh_split[0] = df_cauhinh_split[0].apply(lambda x: x[a+1:-1])

b = df_cauhinh_split[1][0].find('[')
df_cauhinh_split[1] = df_cauhinh_split[1].apply(lambda x: x[b+1:-1])

b = df_cauhinh_split[2][0].find('[')
df_cauhinh_split[2] = df_cauhinh_split[2].apply(lambda x: x[b+1:-1])

b = df_cauhinh_split[3][0].find('[')
df_cauhinh_split[3] = df_cauhinh_split[3].apply(lambda x: x[b+1:-1])

b = df_cauhinh_split[4][0].find('[')
df_cauhinh_split[4] = df_cauhinh_split[4].apply(lambda x: x[b+1:-1])

b = df_cauhinh_split[5][0].find('[')
df_cauhinh_split[5] = df_cauhinh_split[5].apply(lambda x: x[b+1:-1])
df_cauhinh_split = df_cauhinh_split.rename(
    columns = {
        0 : 'cauhinh',
        1 : 'camera_manhinh',
        2 : 'pin_sac',
        3 : 'tien_ich',
        4 : 'ket_noi',
        5 : 'thietke_chatlieu'
    }
)
df_cauhinh_split['thietke_chatlieu'].str.split(r'\),',expand=True)

,0,1,2,3,4
0,"('Thiết kế', 'Nguyên khối'","('Chất liệu', 'Khung nhôm & Mặt lưng kính cườ...","('Kích thước, khối lượng', 'Dài 158.4 mm - Ng...","('Thời điểm ra mắt', '07/2025'","('Hãng', 'Samsung. Xem thông tin hãng')"
1,"('Thiết kế', 'Nguyên khối'","('Chất liệu', 'Khung kim loại & Mặt lưng kính'","('Kích thước, khối lượng', 'Dài 162.2 mm - Ng...","('Thời điểm ra mắt', '03/2025'","('Hãng', 'Samsung. Xem thông tin hãng')"
2,"('Thiết kế', 'Nguyên khối'","('Chất liệu', 'Khung Titan & Mặt lưng kính cư...","('Kích thước, khối lượng', 'Dài 162.8 mm - Ng...","('Thời điểm ra mắt', '01/2025'","('Hãng', 'Samsung. Xem thông tin hãng')"
3,"('Thiết kế', 'Nguyên khối'","('Chất liệu', 'Khung kim loại & Mặt lưng kính'","('Kích thước, khối lượng', 'Dài 162 mm - Ngan...","('Thời điểm ra mắt', '09/2024'","('Hãng', 'Samsung. Xem thông tin hãng')"
4,"('Thiết kế', 'Nguyên khối'","('Chất liệu', 'Khung nhựa & Mặt lưng kính'","('Kích thước, khối lượng', 'Dài 164 mm - Ngan...","('Thời điểm ra mắt', '04/2025'","('Hãng', 'Samsung. Xem thông tin hãng')"
5,"('Thiết kế', 'Nguyên khối'","('Chất liệu', 'Khung & Mặt lưng nhựa'","('Kích thước, khối lượng', 'Dài 167.3 mm - Ng...","('Thời điểm ra mắt', '03/2025'","('Hãng', 'Samsung. Xem thông tin hãng')"
6,"('Thiết kế', 'Nguyên khối'","('Chất liệu', 'Khung nhựa & Mặt lưng kính'","('Kích thước, khối lượng', 'Dài 162.9 mm - Ng...","('Thời điểm ra mắt', '03/2025'","('Hãng', 'Samsung. Xem thông tin hãng')"
7,"('Thiết kế', 'Nguyên khối'","('Chất liệu', 'Khung nhôm & Mặt lưng kính cườ...","('Kích thước, khối lượng', 'Dài 147 mm - Ngan...","('Thời điểm ra mắt', '01/2024'","('Hãng', 'Samsung. Xem thông tin hãng')"
8,"('Thiết kế', 'Nguyên khối'","('Chất liệu', 'Khung nhôm & Mặt lưng kính cườ...","('Kích thước, khối lượng', 'Dài 166.7 (khi mở...","('Thời điểm ra mắt', '07/2025'","('Hãng', 'Samsung. Xem thông tin hãng')"
9,"('Thiết kế', 'Nguyên khối'","('Chất liệu', 'Khung nhôm & Mặt lưng kính cườ...","('Kích thước, khối lượng', 'Dài 146.9 mm - Ng...","('Thời điểm ra mắt', '01/2025'","('Hãng', 'Samsung. Xem thông tin hãng')"


In [53]:
import pandas as pd
import ast

# Load CSV
df = pd.read_csv('product.csv')

# Tên các tab trong 'Cấu hình'
tab_names = ['cauhinh', 'camera_manhinh', 'pin_sac', 'tien_ich', 'ket_noi', 'thietke_chatlieu']

# 1. Split theo ';' để tách các tab
df_tabs = df['Cấu hình'].str.split(';', expand=True)

# 2. Hàm làm sạch chuỗi: lấy đúng phần list of tuples
def clean_tab_string(s):
    if pd.isna(s):
        return '[]'  # list rỗng
    start = s.find('[')
    end = s.rfind(']')
    if start != -1 and end != -1:
        return s[start:end+1]
    else:
        return '[]'

# 3. Chuyển từng tab thành dict
for i, tab in enumerate(tab_names):
    df_tabs[i] = df_tabs[i].apply(clean_tab_string)       # làm sạch chuỗi
    df_tabs[tab] = df_tabs[i].apply(lambda x: dict(ast.literal_eval(x)))  # string -> dict
# 4. Bung dict từng tab thành cột riêng
df_expanded = df.copy()
for tab in tab_names:
    tab_df = df_tabs[tab].apply(pd.Series)
    df_expanded = pd.concat([df_expanded, tab_df], axis=1)

# 5. Xóa cột gốc nếu muốn
df_expanded = df_expanded.drop(columns=['Cấu hình'])

df_expanded['Hãng'] = df_expanded['Hãng'].apply(lambda x: x[:x.find('.')])
df_expanded['Tên'] = df_expanded['Tên'].apply(lambda x: x[len('Điện thoại '):])
# 6. Xem kết quả
thumb = pd.read_csv('thumb_phone.csv')
df_expanded.head()

df_expanded = pd.merge(thumb,df_expanded,left_on='Name',right_on='Tên',how = 'inner')
df_expanded = df_expanded.drop(columns = ['Tên'])
df_expanded.columns = [
    'name', 'thumb', 'price', 'images', 'main_image', 'os',
    'cpu', 'cpu_speed', 'gpu', 'ram',
    'storage', 'available_storage', 'contacts',
    'memory_card', 'rear_camera_resolution', 'rear_camera_video',
    'rear_camera_flash', 'rear_camera_features',
    'front_camera_resolution', 'front_camera_features',
    'screen_tech', 'screen_resolution', 'screen_size',
    'max_brightness', 'screen_protection', 'battery_capacity', 'battery_type',
    'max_charging', 'battery_tech', 'charger_included',
    'security_features', 'special_features', 'waterproof', 'recording',
    'video_formats', 'audio_formats', 'radio', 'mobile_network', 'sim', 'wifi', 'gps',
    'bluetooth', 'ports', 'headphone_jack', 'other_connections',
    'design', 'material', 'dimensions_weight', 'release_date',
    'brand'
]
df_expanded['type'] = 'phone'
# 7. (Tuỳ chọn) xuất CSV
df_expanded.to_csv('product_expanded.csv',index = False)
